In [7]:
import os
import numpy as np
import pandas as pd
import glob
import zipfile
import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import sys


os.getcwd()

'/opt/conda/jupyter/notebooks/지사화 수출유망도 모델'

# 수출 확률 계산을 위해 추출할 데이터는 다음과 같습니다.
-------------------------

## o 수출 데이터를 보유하고 있는 전체 기업들

> 수출국가, 품목, 해당 기업의 2개년 (22년,23년) 수출액, 수출품목수, 수출국가수 등의 정보 

----------------
##  o 만약 기업이 특정 품목, 국가에 대해
##     22년도에 수출 후 23년도에 수출하지 않거나, 23년도부터 새로 수출한 경우
> 수출 데이터가 없는 해당 년도의 정보들만 0으로 입력됩니다
-------------

# 1. 수출 데이터 보유 기업 데이터 추출
-----------------------

### 각 기업별 22,23년도에 수출한 적이 있는 품목들에 대해
### 전체 기업 기준으로 해당 품목들이 22,23년도에 수출된 모든 국가들을 고려했습니다.
-------------------
### 기업번호-품목번호-국가코드 로 각 행이 생성되고
### 각 기업이 특정 품목을 해당 국가에 수출한 적이 있으면 해당 값 조회 후 입력,
### 수출한 적이 없다면 해당 국가 수출 데이터를 0으로 입력했습니다.

In [8]:
trade_file_list = './데이터셋/수출통계'
trade_files = glob.glob(os.path.join(trade_file_list, "*.csv"))

column_types1 = {
    'YEAR' : 'int64',
    'BSNO' : 'category',
    'COUNTRYCD' : 'category',
    'HSCD' : 'category',
    'EXIMAMT' : 'int64'
}

trade_list = [pd.read_csv(file, dtype = column_types1, na_filter=False) for file in trade_files]
combined_trade = pd.concat(trade_list, ignore_index = True)

# 3) 데이터 전처리
## countryname 컬럼삭제 (추후 다시 붙일 예정)
#combined_trade = combined_trade.drop(columns=['COUNTRYNAME'])

# bsno 없는 것 삭제
combined_trade = combined_trade.dropna(subset=['BSNO'])

# 데이터 범위 설정
combined_trade = combined_trade[(combined_trade['YEAR'] >= 2019) & (combined_trade['YEAR'] <= 2024)]

# 나미비아 NA로 바꿔주기
combined_trade.loc[:,'COUNTRYCD'] = combined_trade['COUNTRYCD'].fillna('NA')

# 수출액이 0인 경우 모두 삭제
combined_trade = combined_trade[combined_trade['EXIMAMT'] !=0]

# 특정 사업자번호 삭제
combined_trade = combined_trade[combined_trade['BSNO'] != '1000000000'] # 6243241

In [9]:
# 20~24년까지 개별 데이터프레임 만들기
for year in range(2020, 2024):
    df_c = combined_trade[combined_trade['YEAR'] == year].copy()

    # 1) 파생변수 생성
    df_c['전세계_해당품목_수출액'] = df_c.groupby(['BSNO', 'HSCD'])['EXIMAMT'].transform('sum')
    df_c['전세계_전체품목_수출액'] = df_c.groupby('BSNO')['EXIMAMT'].transform('sum')
    df_c['해당국_전체품목_수출액'] = df_c.groupby(['BSNO', 'COUNTRYCD'])['EXIMAMT'].transform('sum')
    df_c['수출품목수_전세계'] = df_c.groupby('BSNO')['HSCD'].transform('nunique')
    df_c['수출국가수_전체품목'] = df_c.groupby('BSNO')['COUNTRYCD'].transform('nunique')
    df_c['수출품목수_해당국'] = df_c.groupby(['BSNO', 'COUNTRYCD'])['HSCD'].transform('nunique')
    df_c['수출국가수_해당품목'] = df_c.groupby(['BSNO', 'HSCD'])['COUNTRYCD'].transform('nunique')

    df_c = df_c.rename(columns= {'EXIMAMT': '해당국_해당품목_수출액'}) # 컬럼명 변경

    # 2) 내년 수출액 추가 (Y값 생성)
    y = combined_trade[combined_trade['YEAR'] == year+1] # 1033511
    y = y.drop(columns=['YEAR'])
    y = y.rename(columns= {'EXIMAMT': 'NEXT_EXIMAMT'})

    df_c = pd.merge(df_c, y, on =['COUNTRYCD', 'HSCD', 'BSNO'], how='left') # y값 병합
    df_c['NEXT_EXIMAMT'] = df_c['NEXT_EXIMAMT'].fillna(0) # 내년 수출액 없는 경우 0으로 치환
    df_c['RESULT'] = df_c['NEXT_EXIMAMT'].apply(lambda x: 1 if x > 0 else 0 ) # 내년에 수출액이 있으면 1, 없으면 0
    df_c = df_c.drop(columns=['NEXT_EXIMAMT']) # 내년 수출액 컬럼 삭제

    # 3) 작년 수출액 추가
    y_pre = combined_trade[combined_trade['YEAR'] == year-1] # 1033511
    y_pre = y_pre.drop(columns=['YEAR'])
    y_pre = y_pre.rename(columns= {'EXIMAMT': 'PRE_EXIMAMT'})

    df_c = pd.merge(df_c, y_pre, on =['COUNTRYCD', 'HSCD', 'BSNO'], how='left')
    df_c['PRE_EXIMAMT'] = df_c['PRE_EXIMAMT'].fillna(0)
    # 작년에 수출액이 있으면 1 없으면 0
    #df_c['PRE_EXIMAMT'] = df_c['PRE_EXIMAMT'].apply(lambda x: 1 if x > 0 else 0 )

    # 4) HSCD 2단위 / 4단위 컬럼 생성
    df_c['HSCD_2'] =  df_c['HSCD'].astype(str).str[:2]  
    df_c['HSCD_4'] =  df_c['HSCD'].astype(str).str[:4]
    
    # 6) 문자열로 변환하여 통일
    df_c['HSCD'] = df_c['HSCD'].astype(str)
    df_c['HSCD_2'] = df_c['HSCD_2'].astype(str)
    df_c['HSCD_4'] = df_c['HSCD_4'].astype(str)
    df_c['COUNTRYCD'] = df_c['COUNTRYCD'].astype(str)
    df_c['BSNO'] = df_c['BSNO'].astype(str)

    # 7) COUNTRYCD 100건 미만의 적은 국가 제거
    COUNTRYCD_CNT = df_c['COUNTRYCD'].value_counts()
    valid_COUNTRYCD = COUNTRYCD_CNT[COUNTRYCD_CNT >= 100].index
    df_c = df_c[df_c['COUNTRYCD'].isin(valid_COUNTRYCD)]

    
    # 8) 데이터 추가
    df_c.to_csv(f'./전처리데이터/stack_value_{year-2000}_test.csv', index=False)
    


In [10]:
df_test_22=pd.read_csv('./전처리데이터/stack_value_22_test.csv', dtype={'BSNO' : str, 'HSCD' : str}, na_filter=False)
df_test_23=pd.read_csv('./전처리데이터/stack_value_23_test.csv', dtype={'BSNO' : str, 'HSCD' : str}, na_filter=False)

df_test=pd.concat([df_test_22, df_test_23], axis=0)
df_test.drop(columns=['RESULT', 'PRE_EXIMAMT', 'HSCD_2', 'HSCD_4'], inplace=True)

bsno_all=df_test['BSNO'].unique()

In [11]:
df_test

,YEAR,COUNTRYCD,HSCD,BSNO,해당국_해당품목_수출액,전세계_해당품목_수출액,전세계_전체품목_수출액,해당국_전체품목_수출액,수출품목수_전세계,수출국가수_전체품목,수출품목수_해당국,수출국가수_해당품목
0,2022,AE,010619,4728600892,318,15005,15005,318,1,15,1,15
1,2022,AE,010619,5418101760,80666,214913,214913,80666,1,5,1,5
2,2022,AE,010619,7617600319,252,50306,50306,252,1,4,1,4
3,2022,AE,020321,6168624854,19661,19661,2294827,319761,97,16,51,1
4,2022,AE,020322,6168624854,4837,4837,2294827,319761,97,16,51,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1121773,2023,ZZ,950300,3210401603,150,160665,2192683,345,137,127,2,51
1121774,2023,ZZ,950590,1208771116,46,108156,33427621,84,115,89,2,36
1121775,2023,ZZ,950639,3210401603,195,3559,2192683,345,137,127,2,7
1121776,2023,ZZ,950824,3148618854,329728,4863801,9119956,449728,72,55,5,46


# 22,23년도 전체에서 수출한 데이터가 하나라도 존재하는 기업 개수 : 121844

In [4]:
len(bsno_all)

121844

## 전체 수출 데이터 (22,23년) 에서 '수출 품목별 수출 국가 명단'을 저장합니다.
> 이후 데이터에서 기업의 특정 수출 품목에 대해, 

> 수출한 적이 있는 국가들 뿐만 아니라, 다른 국가들의 예측 확률도 구해 **유망 국가 리스트**를 보여주기 위해 필요합니다. 

In [5]:
df_test_hscd_cty=df_test[['HSCD', 'COUNTRYCD']]
df_test_hscd_cty.drop_duplicates(inplace=True)

hscd_cty_dict={}

for _, row in tqdm(df_test_hscd_cty.iterrows()):
    key=row['HSCD']
    value=row['COUNTRYCD']
    if key not in hscd_cty_dict:
        hscd_cty_dict[key]=[]
    if value not in hscd_cty_dict[key]:
        hscd_cty_dict[key].append(value)  
        
print('22,23년도 데이터의 총 수출 품목 개수 : ', len(hscd_cty_dict))

/opt/common-library/conda/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
178499it [00:06, 29187.80it/s]

22,23년도 데이터의 총 수출 품목 개수 :  5122


## 22, 23년도 각각의 수출 데이터 에서 '기업의 수출 품목별 데이터'를 저장합니다.
> 이후 데이터에서 기업의 수출 품목별 해당되는 

> **'전세계_해당품목_수출액', '수출국가수_해당품목' 값**을 입력하기 위해 필요합니다. 

In [6]:
bsno_hscd_agg_22_dict={}

for _, row in tqdm(df_test_22.iterrows()):
    key=row['BSNO']+'_'+row['HSCD']
    value=[row['전세계_해당품목_수출액'], row['수출국가수_해당품목']]
    if key not in bsno_hscd_agg_22_dict:
        bsno_hscd_agg_22_dict[key]=[]
        bsno_hscd_agg_22_dict[key].extend(value)  

1060590it [00:41, 25765.88it/s]


In [7]:
bsno_hscd_agg_23_dict={}

for _, row in tqdm(df_test_23.iterrows()):
    key=row['BSNO']+'_'+row['HSCD']
    value=[row['전세계_해당품목_수출액'], row['수출국가수_해당품목']]
    if key not in bsno_hscd_agg_23_dict:
        bsno_hscd_agg_23_dict[key]=[]
        bsno_hscd_agg_23_dict[key].extend(value)  

1121778it [00:44, 25494.55it/s]


## 22, 23년도 각각의 수출 데이터 에서 '기업의 수출 국가별 데이터'를 저장합니다.
> 이후 데이터에서 기업의 수출 국가별 해당되는

> **'해당국_전체품목_수출액', '수출품목수_해당국'**을 입력하기 위해 필요합니다. 

In [8]:
bsno_cty_agg_22_dict={}

for _, row in tqdm(df_test_22.iterrows()):
    key=str(row['BSNO'])+'_'+str(row['COUNTRYCD'])
    value=[row['해당국_전체품목_수출액'], row['수출품목수_해당국']]
    if key not in bsno_cty_agg_22_dict:
        bsno_cty_agg_22_dict[key]=[]
        bsno_cty_agg_22_dict[key].extend(value)  

1060590it [00:41, 25577.31it/s]


In [9]:
bsno_cty_agg_23_dict={}

for _, row in tqdm(df_test_23.iterrows()):
    key=row['BSNO']+'_'+row['COUNTRYCD']
    value=[row['해당국_전체품목_수출액'], row['수출품목수_해당국']]
    if key not in bsno_cty_agg_23_dict:
        bsno_cty_agg_23_dict[key]=[]
        bsno_cty_agg_23_dict[key].extend(value)  

1121778it [00:44, 25484.73it/s]


## 22, 23년도 각각의 수출 데이터 에서 '기업별 데이터'를 저장합니다.
> 이후 데이터에서 기업별 해당되는 

> **'전세계_전체품목_수출액', '수출품목수_전세계', '수출국가수_전체품목'** 을 입력하기 위해 필요합니다. 

In [10]:
bsno_agg_22_dict={}

for _, row in tqdm(df_test_22.iterrows()):
    key=row['BSNO']
    value=[row['전세계_전체품목_수출액'], row['수출품목수_전세계'], row['수출국가수_전체품목']]
    if key not in bsno_agg_22_dict:
        bsno_agg_22_dict[key]=[]
        bsno_agg_22_dict[key].extend(value)  

1060590it [00:40, 26391.52it/s]


In [11]:
bsno_agg_23_dict={}

for _, row in tqdm(df_test_23.iterrows()):
    key=row['BSNO']
    value=[row['전세계_전체품목_수출액'], row['수출품목수_전세계'], row['수출국가수_전체품목']]
    if key not in bsno_agg_23_dict:
        bsno_agg_23_dict[key]=[]
        bsno_agg_23_dict[key].extend(value)  

1121778it [00:42, 26411.58it/s]


## 22, 23년도 각각의 수출 데이터 에서 '기업의 수출 품목별 수출 국가 명단'을 저장합니다.
> 이후 데이터에서 기업의 수출 품목별 해당되는 **수출 국가를 제외한 국가에 대해서만 새로운 행**을 추가해, 

> 수출하지 않은 다른 국가들의 예측 확률을 구해 **유망 국가 리스트**를 만들기 위해 필요합니다.

In [12]:
bsno_hscd_cty_dup_22_dict={}


for _, row in tqdm(df_test_22.iterrows()):
    key=row['BSNO']+'_'+row['HSCD']
    value=row['COUNTRYCD']
    if key not in bsno_hscd_cty_dup_22_dict:
        bsno_hscd_cty_dup_22_dict[key]=[value]
    if value not in bsno_hscd_cty_dup_22_dict[key]:
        bsno_hscd_cty_dup_22_dict[key].append(value)   

1060590it [00:40, 26497.45it/s]


In [13]:
bsno_hscd_cty_dup_23_dict={}


for _, row in tqdm(df_test_23.iterrows()):
    key=str(row['BSNO'])+'_'+str(row['HSCD'])
    value=row['COUNTRYCD']
    if key not in bsno_hscd_cty_dup_23_dict:
        bsno_hscd_cty_dup_23_dict[key]=[value]
    if value not in bsno_hscd_cty_dup_23_dict[key]:
        bsno_hscd_cty_dup_23_dict[key].append(value)   

1121778it [00:43, 26061.03it/s]


## 데이터가 있는 기업들에 대해서만, 22, 23년도 각각의 수출 데이터를 만듭니다.


In [14]:
def create_export_data(year):
    
    # 입력된 연도의 테스트 데이터를 불러와 추론에 필요한 열만 남기고 제거
    
    df_test_new=globals()[f'df_test_{year}'].copy()
    df_test_new.drop(columns=['RESULT', 'PRE_EXIMAMT', 'HSCD_4'], inplace=True)
    
    # 해당 년도에서, 특정 기업이 수출하고 있는 품목의 명단을 정리 
    
    df_test_hscd_bsno=df_test_new[['HSCD', 'BSNO']]
    df_test_hscd_bsno.drop_duplicates(inplace=True)

    bsno_hscd_dict={}

    for _, row in tqdm(df_test_hscd_bsno.iterrows()):
        key=row['BSNO']
        value=row['HSCD']
        if key not in bsno_hscd_dict:
            bsno_hscd_dict[key]=[]
        if value not in bsno_hscd_dict[key]:
            bsno_hscd_dict[key].append(value)

    new_df=pd.DataFrame()
    j=0
    
    # 각 기업과 그 기업이 수출하고 있는 품목의 명단으로 반복해 실행
    
    for i, (bsno, hscds) in tqdm(enumerate(bsno_hscd_dict.items())):

        all_hscd_world_list=[]
        all_hscd_cty_list=[]
        cnt_hscd_world_list=[]
        cnt_cty_all_hscd_list=[]
        hscd_world_list=[]
        cnt_cty_hscd_list=[]
        cnt_hscd_cty_list=[]
        hscd_list=[]
        cty_list=[]

        # 특정 기업이 해당 년도에 같은 값을 가지는 값들
        # '전세계_전체품목_수출액', '수출품목수_전세계', '수출국가수_전체품목'
        
        all_hscd_world=globals()[f'bsno_agg_{year}_dict'][f'{bsno}'][0] # bsno 같으면 동일값
        cnt_hscd_world=globals()[f'bsno_agg_{year}_dict'][f'{bsno}'][1] # bsno 같으면 동일값
        cnt_cty_all_hscd=globals()[f'bsno_agg_{year}_dict'][f'{bsno}'][2] # bsno 같으면 동일값
        
        
        for hscd in hscds:

            # 특정 기업의 특정 수출 품목이 해당 년도에 같은 값을 가지는 값들
            # '전세계_해당품목_수출액', '수출국가수_해당품목'
            
            hscd_world=globals()[f'bsno_hscd_agg_{year}_dict'][f'{bsno}_{hscd}'][0] # bsno, hscd 같으면 동일값
            cnt_cty_hscd=globals()[f'bsno_hscd_agg_{year}_dict'][f'{bsno}_{hscd}'][1] # bsno, hscd 같으면 동일값

            cty_len=0
            
            # 특정 품목이 수출되고 있는 국가들 중, 
            # 해당 기업이 이미 수출한 국가가 아닌 다른 국가 (유망 수출국 후보지) 들에 대해 반복
            
            for cty in [c for c in hscd_cty_dict[hscd] if c not in globals()[f'bsno_hscd_cty_dup_{year}_dict'][f'{bsno}_{hscd}']]:
                
                # 만약, 기업이 해당 년도, 해당 국가에 수출한 품목이 한개라도 있다면
                # '해당국_전체품목_수출액', '수출품목수_해당국' 값을 찾아 넣어줌
                
                try:
                    cnt_hscd_cty=globals()[f'bsno_cty_agg_{year}_dict'][f'{bsno}_{cty}'][1]  # bsno, hscd, cty 같으면 동일값
                    all_hscd_cty=globals()[f'bsno_cty_agg_{year}_dict'][f'{bsno}_{cty}'][0] # bsno, hscd, cty 같으면 동일값


                    all_hscd_cty_list.append(all_hscd_cty)
                    cnt_cty_hscd_list.append(cnt_cty_hscd)
                    cnt_hscd_cty_list.append(cnt_hscd_cty)
                    cty_list.append(cty)
                    cty_len+=1
                    
                # 만약, 기업이 해당 년도, 해당 국가에 수출한 품목이 한 개도 없다면
                # 해당국_전체품목_수출액', '수출품목수_해당국' 값은 모두 0
                
                except:
                    cnt_hscd_cty=0
                    all_hscd_cty=0


                    all_hscd_cty_list.append(all_hscd_cty)
                    cnt_cty_hscd_list.append(cnt_cty_hscd)
                    cnt_hscd_cty_list.append(cnt_hscd_cty)
                    cty_list.append(cty)
                    cty_len+=1
                    continue

            hscd_world_list.extend([hscd_world]*cty_len)
            hscd_list.extend([hscd]*cty_len)

        all_hscd_world_list=[all_hscd_world]*len(cty_list)
        cnt_hscd_world_list=[cnt_hscd_world]*len(cty_list)
        cnt_cty_all_hscd_list=[cnt_cty_all_hscd]*len(cty_list)

        # 각 기업에서 해당 년도에 수출하고 있는 (모든 품목 * 각 품목이 수출되는 모든 국가) 조합에 대해
        # 가지고 있는 데이터의 값을 입력하고, 없으면 0으로 둔 데이터프레임 생성
        
        ## 데이터 추가 속도를 향상시키기 위해, 행을 하나하나 붙이지 않고
        ## 각 기업단위로 데이터프레임을 만들되 , list를 이용해 열 데이터를 한번에 할당시킴
        
        append_df = pd.DataFrame({'COUNTRYCD': cty_list, 
                                   'HSCD':hscd_list, 
                                   'BSNO' : [bsno]*len(cty_list),
                                    '해당국_해당품목_수출액' : [0]*len(cty_list), 
                                   '전세계_해당품목_수출액' : hscd_world_list, 
                                   '전세계_전체품목_수출액' : all_hscd_world_list,
                                    '해당국_전체품목_수출액' : all_hscd_cty_list, 
                                   '수출품목수_전세계' : cnt_hscd_world_list, 
                                   '수출국가수_전체품목' : cnt_cty_all_hscd_list,
                                    '수출품목수_해당국' : cnt_hscd_cty_list, 
                                   '수출국가수_해당품목': cnt_cty_hscd_list })

        new_df=pd.concat([new_df,append_df] , axis=0)

        # 각 기업의 데이터를 모은 데이터프레임을 합치는 도중
        # 합쳐진 데이터프레임의 크기가 너무 큰 경우 concat 계산 속도를 위해 초기화
        ## 데이터프레임의 크기가 100MB 이상일 경우 concat 속도가 현저히 느려져 csv로 저장 후 초기화
        
        if sys.getsizeof(new_df)>100*1024*1024:
            new_df.to_csv(f'./전처리데이터/test_set/test_{year}_all_{j}.csv', index=False)
            new_df=pd.DataFrame()
            print('new_df refreshed!')
            j+=1

        if i%1000==0:
            print('current data size :',  sys.getsizeof(new_df))  
            
    new_df.to_csv(f'./전처리데이터/test_set/test_{year}_all_{j}.csv', index=False)
    df_test_new=pd.concat([df_test_new] +[pd.read_csv(f'./전처리데이터/test_set/test_{year}_all_{k}.csv') for k in range(j+1)], axis=0)
    df_test_new.to_csv(f'./전처리데이터/stack_value_{year}_test_all.csv', index=False)

In [15]:
create_export_data(22)
create_export_data(23)

550061it [00:18, 29604.31it/s]
10it [00:00, 98.86it/s]

current data size : 3686


163it [00:09, 15.65it/s]

new_df refreshed!


320it [00:17, 11.42it/s]

...

83053it [1:06:03, 26.38it/s]

new_df refreshed!


84003it [1:06:37, 15.75it/s]

current data size : 69419491


84403it [1:07:02, 14.08it/s]

## 전체 수출 데이터 (22,23년) 에서 각 기업이 해당 품목에 대해 
## 기존에 수출하지 않은 국가들에 대한 행도 추가한 데이터셋입니다.

In [ ]:
# 2022, 2023 데이터를 열 방향으로 이어붙임
df_p=pd.read_csv('./전처리데이터/stack_value_22_test_all.csv')
df_n=pd.read_csv('./전처리데이터/stack_value_23_test_all.csv')

# 학습시킨 데이터와 동일한 형태로 자료형 변환
df_n['BSNO']=df_n['BSNO'].astype(float).astype(int).astype(str)
df_n['HSCD']=df_n['HSCD'].astype(float).astype(int).astype(str)
df_p['BSNO']=df_p['BSNO'].astype(float).astype(int).astype(str)
df_p['HSCD']=df_p['HSCD'].astype(float).astype(int).astype(str)


# 데이터 합산
df_combined = pd.merge(df_n, df_p, on = ['BSNO', 'HSCD', 'COUNTRYCD'], how= 'outer', suffixes = ('_n', '_p')) # 1139264


# 22년에 수출이 없었던 값 or 23년에 수출하지 않는 품목은 값을 0으로 채우기
df_combined = df_combined.fillna(0)
df_combined.to_csv('./전처리데이터/stack_value_test_all.csv', index=False)

In [3]:
df_combined

,YEAR_n,COUNTRYCD,HSCD,BSNO,해당국_해당품목_수출액_n,전세계_해당품목_수출액_n,전세계_전체품목_수출액_n,해당국_전체품목_수출액_n,수출품목수_전세계_n,수출국가수_전체품목_n,...,YEAR_p,해당국_해당품목_수출액_p,전세계_해당품목_수출액_p,전세계_전체품목_수출액_p,해당국_전체품목_수출액_p,수출품목수_전세계_p,수출국가수_전체품목_p,수출품목수_해당국_p,수출국가수_해당품목_p,HSCD_2_p
0,2023.0,AE,10619,4728600892,739.0,22524.0,22524.0,739.0,1.0,13.0,...,2022.0,318.0,15005.0,15005.0,318.0,1.0,15.0,1.0,15.0,1.0
1,2023.0,AE,10619,5418101760,8790.0,127951.0,127951.0,8790.0,1.0,10.0,...,2022.0,80666.0,214913.0,214913.0,80666.0,1.0,5.0,1.0,5.0,1.0
2,2023.0,AE,10619,8212001739,1000.0,70012.0,70012.0,1000.0,1.0,7.0,...,0.0,0.0,55117.0,55117.0,0.0,1.0,7.0,0.0,7.0,0.0
3,2023.0,AE,20322,6168624854,3531.0,4841.0,2172627.0,151874.0,76.0,19.0,...,2022.0,4837.0,4837.0,2294827.0,319761.0,97.0,16.0,51.0,1.0,2.0
4,2023.0,AE,20329,6168624854,50899.0,93590.0,2172627.0,151874.0,76.0,19.0,...,2022.0,58984.0,58984.0,2294827.0,319761.0,97.0,16.0,51.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75360856,0.0,SG,730519,6048108837,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37700.0,37700.0,0.0,1.0,1.0,0.0,1.0,0.0
75360857,0.0,TH,730519,6048108837,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37700.0,37700.0,0.0,1.0,1.0,0.0,1.0,0.0
75360858,0.0,VN,730519,6048108837,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37700.0,37700.0,0.0,1.0,1.0,0.0,1.0,0.0
75360859,0.0,CN,730519,6048108837,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37700.0,37700.0,0.0,1.0,1.0,0.0,1.0,0.0


# 모델 학습 (기존에 학습시킨 모델이 있다면 실행하지 않아도 됨)

In [ ]:
# 수출통계 데이터 합산
trade_file_list = './데이터셋/수출통계'
trade_files = glob.glob(os.path.join(trade_file_list, "*.csv"))

column_types1 = {
    'YEAR' : 'int64',
    'BSNO' : 'category',
    'COUNTRYCD' : 'category',
    'HSCD' : 'category',
    'EXIMAMT' : 'int64'
}

trade_list = [pd.read_csv(file, dtype = column_types1) for file in trade_files]
combined_trade = pd.concat(trade_list, ignore_index = True)

# bsno 없는 것 삭제
combined_trade = combined_trade.dropna(subset=['BSNO'])

# 데이터 범위 설정
combined_trade = combined_trade[(combined_trade['YEAR'] >= 2020) & (combined_trade['YEAR'] <= 2024)]

# 나미비아 NA로 바꿔주기
combined_trade.loc[:,'COUNTRYCD'] = combined_trade['COUNTRYCD'].fillna('NA')

# 수출액이 0인 경우 모두 삭제
combined_trade = combined_trade[combined_trade['EXIMAMT'] !=0]

# 특정 사업자번호 삭제
combined_trade = combined_trade[combined_trade['BSNO'] != '1000000000'] # 5215219

# 2개년 수출기업/수출품목 리스트
yr22 = combined_trade[combined_trade['YEAR'] == 2022]
yr21 = combined_trade[combined_trade['YEAR'] == 2021]

yr22 = yr22.drop(columns = ['YEAR', 'EXIMAMT']).drop_duplicates()
yr21 = yr21.drop(columns = ['YEAR', 'EXIMAMT']).drop_duplicates()

yr_total = pd.merge(yr22, yr21, on = ['BSNO', 'HSCD', 'COUNTRYCD'], how= 'outer') # 1575467

# 연도별 데이터 생성
yr = 2022

for year in range(yr-1, yr+1):
    df_c = combined_trade[combined_trade['YEAR'] == year].copy() # 1033511
    df_c = pd.merge(yr_total, df_c, on = ['BSNO', 'HSCD', 'COUNTRYCD'], how = 'left')

    df_c['YEAR'] = df_c['YEAR'].fillna(year)
    df_c['EXIMAMT'] = df_c['EXIMAMT'].fillna(0)

    # 파생변수 생성
    df_c['전세계_해당품목_수출액'] = df_c.groupby(['BSNO', 'HSCD'])['EXIMAMT'].transform('sum')
    df_c['전세계_전체품목_수출액'] = df_c.groupby('BSNO')['EXIMAMT'].transform('sum')
    df_c['해당국_전체품목_수출액'] = df_c.groupby(['BSNO', 'COUNTRYCD'])['EXIMAMT'].transform('sum')
    df_c['수출품목수_전세계'] = df_c.groupby('BSNO')['HSCD'].transform('nunique')
    df_c['수출국가수_전체품목'] = df_c.groupby('BSNO')['COUNTRYCD'].transform('nunique')
    df_c['수출품목수_해당국'] = df_c.groupby(['BSNO', 'COUNTRYCD'])['HSCD'].transform('nunique')
    df_c['수출국가수_해당품목'] = df_c.groupby(['BSNO', 'HSCD'])['COUNTRYCD'].transform('nunique')
    
    # 동적으로 변수 이름 생성
    sh_yr = str(year)[-2:]
    globals()[f'df_{sh_yr}'] = df_c
    
# 데이터 합산
df_n = globals()[f'df_{str(yr)[-2:]}'] # 22년 (now)
df_p = globals()[f'df_{str(yr-1)[-2:]}'] # 21년 (previous)

# YEAR 삭제
df_n = df_n.drop(columns = 'YEAR') # 1575467
df_p = df_p.drop(columns = 'YEAR') # 1575467

# 컬럼명 변경
df_n = df_n.rename(columns= {'EXIMAMT': '해당국_해당품목_수출액'}) 
df_p = df_p.rename(columns= {'EXIMAMT': '해당국_해당품목_수출액'}) 

# 데이터 합산
df_combined = pd.merge(df_n, df_p, on = ['BSNO', 'HSCD', 'COUNTRYCD'], how= 'outer', suffixes = ('_n', '_p')) # 1551620

# 작년에 수출이 없었던 값 0으로 채우기
df_combined = df_combined.fillna(0)

# Y값 생성
y = combined_trade[combined_trade['YEAR'] == yr+1] # 1033511
y = y.drop(columns=['YEAR'])
y = y.rename(columns= {'EXIMAMT': 'NEXT_EXIMAMT'})

# y값 병합
df_combined = pd.merge(df_combined, y, on =['COUNTRYCD', 'HSCD', 'BSNO'], how='left')

# 내년 수출액 없는 경우 0으로 치환
df_combined['NEXT_EXIMAMT'] = df_combined['NEXT_EXIMAMT'].fillna(0)

# 내년에 수출액이 있으면 1, 없으면 0
df_combined['RESULT'] = df_combined['NEXT_EXIMAMT'].apply(lambda x: 1 if x > 0 else 0 )
df_combined = df_combined.drop(columns=['NEXT_EXIMAMT']) #1551620

# COUNTRYCD 100건 미만의 적은 국가 제거 (1054594)
COUNTRYCD_CNT = df_combined['COUNTRYCD'].value_counts()
valid_COUNTRYCD = COUNTRYCD_CNT[COUNTRYCD_CNT >= 100].index
df_sorted = df_combined[df_combined['COUNTRYCD'].isin(valid_COUNTRYCD)]

# 문자열로 변환하여 통일
df_sorted['HSCD'] = df_sorted['HSCD'].astype(str)
df_sorted['COUNTRYCD'] = df_sorted['COUNTRYCD'].astype(str)
df_sorted['BSNO'] = df_sorted['BSNO'].astype(str)

# 최종 데이터셋
fin_train = df_sorted # 1573454

# BSNO 삭제 
fin_train = fin_train.drop(columns = ['BSNO'])

###################
# 학습데이터
###################
# 입력 데이터와 타겟 변수 설정 
X_train = fin_train.drop(columns=['RESULT'])
y_train = fin_train['RESULT']


# 로그 스케일링 (수출액만 log변환 / 국가수 및 품목수는 scaling 제외)
cols_to_scale = X_train.columns[[2, 3, 4, 5, 10, 11, 12, 13]] # BSNO 삭제
X_train[cols_to_scale] = X_train[cols_to_scale].apply(lambda x:np.log1p(x))


# CatBoost 모델 정의 
# [참고] 확률 0.5이상이면 1, 미만이면 0
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6,
                           cat_features=[0, 1], # BSNO 삭제
                           verbose=False)

# 학습
model.fit(X_train, y_train)

joblib.dump(model, './final_model.pkl')



# 미리 학습된 모델에 데이터를 넣고 확률을 계산합니다.

In [6]:
# 미리 학습된 모델 불러오기

model=joblib.load('./final_model.pkl')

final_test=pd.read_csv('./전처리데이터/stack_value_test_all.csv')

# 사업자 번호 데이터는 추후 대시보드 조회시 필요하므로 따로 저장

final_test_bsno=final_test['BSNO'].copy()
final_test['HSCD']=final_test['HSCD'].apply(str)

# 추론에 필요없는 데이터 열은 제거

final_test.drop(columns=['BSNO','HSCD_2_n', 'YEAR_p', 'YEAR_n', 'HSCD_2_p'], inplace=True)
cols_to_scale = final_test.columns[[2, 3, 4, 5, 10, 11, 12, 13]] # BSNO 삭제

# 학습시와 동일한 방법으로 데이터 스케일링 (np.log1p)
final_test[cols_to_scale] = final_test[cols_to_scale].apply(lambda x:np.log1p(x))

y_pred_proba=model.predict_proba(final_test)
y_pred_proba

array([[0.44013877, 0.55986123],
       [0.39554754, 0.60445246],
       [0.71561148, 0.28438852],
       ...,
       [0.97130706, 0.02869294],
       [0.97445252, 0.02554748],
       [0.98499074, 0.01500926]])

In [7]:
final_test.columns

Index(['COUNTRYCD', 'HSCD', '해당국_해당품목_수출액_n', '전세계_해당품목_수출액_n',
       '전세계_전체품목_수출액_n', '해당국_전체품목_수출액_n', '수출품목수_전세계_n', '수출국가수_전체품목_n',
       '수출품목수_해당국_n', '수출국가수_해당품목_n', '해당국_해당품목_수출액_p', '전세계_해당품목_수출액_p',
       '전세계_전체품목_수출액_p', '해당국_전체품목_수출액_p', '수출품목수_전세계_p', '수출국가수_전체품목_p',
       '수출품목수_해당국_p', '수출국가수_해당품목_p'],
      dtype='object')

In [8]:
# (수출 실패 확률, 수출 성공 확률)로 이루어진 데이터 중
# 수출 성공 확률만 추출해 최종 데이터셋에 concat

pred_proba=pd.DataFrame({'Success' : [y for (n,y) in y_pred_proba]})
final_df=pd.concat([final_test[['COUNTRYCD', 'HSCD']], final_test_bsno, pred_proba], axis=1)

In [9]:
final_df

,COUNTRYCD,HSCD,BSNO,Success
0,AE,10619,4728600892,0.559861
1,AE,10619,5418101760,0.604452
2,AE,10619,8212001739,0.284389
3,AE,20322,6168624854,0.455069
4,AE,20329,6168624854,0.635991
...,...,...,...,...
75360856,SG,730519,6048108837,0.029268
75360857,TH,730519,6048108837,0.028897
75360858,VN,730519,6048108837,0.028693
75360859,CN,730519,6048108837,0.025547


In [10]:
final_df.to_csv('./전처리데이터/final_proba_all.csv', index=False)